In [17]:
import numpy as np
import csv
import pandas as pd

In [18]:
## 读取csv文件
data_train_origin = []
train = csv.reader(open('C:/Users/Administrator/Desktop/python_practice/Fudan_practice/text_classification_ML/train.tsv', 'rt'))
for row in train:
    data_train_origin.append(row)

data_test_origin = []
train = csv.reader(open('C:/Users/Administrator/Desktop/python_practice/Fudan_practice/text_classification_ML/test.tsv', 'rt'))
for row in train:
    data_test_origin.append(row)

In [3]:
## 数据的分割
data_train = []
for item in data_train_origin:
    if (len(item)>0):
        for j in range(1,len(item)):
            item[0]+=item[j]
    data_train.append(item[0].split('\t'))

data_test = []
for item in data_test_origin:
    if (len(item)>0):
        for j in range(1,len(item)):
            item[0]+=item[j]
    data_test.append(item[0].split('\t'))
    

In [4]:
## 删除标题
data_train.remove(data_train[0])
data_test.remove(data_test[0])



In [5]:
## 简单预处理
for i in range(len(data_train)):
    data_train[i][0] = int(data_train[i][0])
    data_train[i][1] = int(data_train[i][1])
    data_train[i][2] = data_train[i][2].replace('  ',' ')
    data_train[i][3] = int(data_train[i][3])

for i in range(len(data_test)):
    data_test[i][0] = int(data_test[i][0])
    data_test[i][1] = int(data_test[i][1])
    data_test[i][2] = data_test[i][2].replace('  ',' ')

In [6]:
## 建立词袋模型来表征句子

# 切分单词
tokenized_lines = []
for item in data_train:
    tokenized_lines.append(item[2].split(' '))


In [7]:
# 去除标点，替换大小写
punctuation = [",", ":", ";", ".", "'", '"', "’", "?", "/", "-", "+", "&", "(", ")"]
clean_tokenized = []
for item in tokenized_lines:
    tokens = []
    for token in item:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc, "")
        tokens.append(token)
    clean_tokenized.append(tokens)


In [8]:
# 生成原始的词汇集合,剔除了只出现过一次的token
unique_tokens = []
single_tokens = []
for tokens in clean_tokenized:
    for token in tokens:
        if token not in single_tokens:
            single_tokens.append(token)
        elif token in single_tokens and token not in unique_tokens:
            unique_tokens.append(token)



In [ ]:
# 将二维的tokens转成一维tokens
all_tokens = []
for items in clean_tokenized:
    for item in items:
        all_tokens.append(item.lower())

In [ ]:
# 统计每个词出现的频率
dic = {}
for token in unique_tokens:
    dic[token] = all_tokens.count(token)

In [ ]:
# 筛选特征在10-100之间
tokens = []
freq = []
for i in range(len(dic)):
    if dic.values()[i]>=10 and dic.values()[i]<=100:
        tokens.append(dic.keys()[i])
        freq.append(dic.values()[i])


In [ ]:
# 将每个句子按照BOW特征空间进行表征
representations =[]
for seq in clean_tokenized:
    rep = []
    for token in tokens:
        if token in seq:
            rep.append(1)
        else:
            rep.append(0)
    representations.append(rep)

In [ ]:
## 定义x，y
x = representations
y = []
for item in data_train:
    y.append(item[3])

In [ ]:
## y生成one-hot vector
classes = np.max(y) + 1
y = np.eye(classes)[y]

In [ ]:
## 划分集合
n_train = int(len(x)*0.8)
n_test = int(len(x)*0.1)
n_dev = int(len(x) - n_train -n_test)

import random
index_train = random.sample(range(len(x)),n_train)
index_dev = random.sample(list(set(range(len(x)))^set(index_train)) ,n_dev)
index_test = list(set(list(set(range(len(x)))^set(index_train)))^set(index_dev)) 

x_train = []
y_train = []
for index in index_train:
    x_train.append(x[index])
    y_train.append(y[index])
    
x_dev = []
y_dev = []
for index in index_dev:
    x_dev.append(x[index])
    y_dev.append(y[index])

x_test = []
y_test = []
for index in index_test:
    x_test.append(x[index])
    y_test.append(y[index])

x_train = np.array(x_train)
y_train = np.array(y_train)

x_dev = np.array(x_dev)
y_dev = np.array(y_dev)

x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
## Logistic model
def logit(x_train,y_train,x_dev,y_dev,W,b,alpha,lam):
    y_ = 1/(1+np.exp(np.dot(x_train,W)+b))
    y__dev = 1/(1+np.exp(np.dot(x_dev,W)+b))
    loss = -np.average(y_train*np.log(y_)+(1-y_train)*np.log(1-y_))+lam/2*np.sum(W**2)
    loss_dev = -np.average(y_dev*np.log(y__dev)+(1-y_dev)*np.log(1-y__dev))+lam/2*np.sum(W**2)
    dloss_dW = -np.dot(x_train.T,(y_train-y_))/len(x_train)+lam*W
    dloss_db = np.average(y_train-y_,axis = 0)
    W_new = W - alpha*dloss_dW
    b_new = b - alpha*dloss_db
    return loss,loss_dev, W_new, b_new

In [ ]:
## 初始化参数，logistic model可以全0初始化
W = np.zeros([np.shape(x_train)[1],np.shape(y_train)[1]])
b = np.zeros([1,np.shape(y_train)[1]])

In [ ]:
## 批量梯度下降法：
def bat_gradesc(x_train,y_train,x_dev,y_dev,W,b,alpha,lam):
    losses = []
    losses_dev = []
    iteration = 0
    num = 0
    while (num<10 and num < 5000):
        loss,loss_dev, W, b = logit(x_train,y_train,x_dev,y_dev,W,b,alpha,lam)
        losses.append(loss)
        losses_dev.append(loss_dev)
        iteration += 1
        if  (len(losses_dev)>=2):
            if (loss_dev-losses_dev[-2]>0):
                num += 1
            else:
                num = 0
        print(str(iteration)+' iteration:')
        print(num)
    return losses, losses_dev, W, b, iteration

In [ ]:
## 随机梯度下降法：
def st_gradesc(x_train,y_train,x_dev,y_dev,W,b,alpha,lam):
    losses = []
    losses_dev = []
    iteration = 0
    num = 0
    while (num<10 and num < 5000):
        rand = random.randint(0,len(x_train))
        x_rand = np.array(x_train[rand])
        y_rand = np.array(y_train[rand])
        loss,loss_dev, W, b = logit(x_rand,y_rand,x_dev,y_dev,W,b,alpha,lam)
        losses.append(loss)
        losses_dev.append(loss_dev)
        iteration += 1
        if  (len(losses_dev)>=2):
            if (loss_dev-losses_dev[-2]>0):
                num += 1
            else:
                num = 0
        print(str(iteration)+' iteration:')
        print(num)
    return losses, losses_dev, W, b, iteration   

In [ ]:
## 小批量梯度下降法：
def st_gradesc(x_train,y_train,x_dev,y_dev,W,b,alpha,lam):
    losses = []
    losses_dev = []
    iteration = 0
    num = 0
    while (num<10 and num < 5000):
        x_rand = np.array(random.sample(list(x_train),100))
        y_rand = np.array(random.sample(list(y_train),100))
        loss,loss_dev, W, b = logit(x_rand,y_rand,x_dev,y_dev,W,b,alpha,lam)
        losses.append(loss)
        losses_dev.append(loss_dev)
        iteration += 1
        if  (len(losses_dev)>=2):
            if (loss_dev-losses_dev[-2]>0):
                num += 1
            else:
                num = 0
        print(str(iteration)+' iteration:')
        print(num)
    return losses, losses_dev, W, b, iteration 

In [ ]:
## 运行迭代
losses, losses_dev, W, b, iteration = bat_gradesc(x_train,y_train,x_dev,y_dev,W,b,0.01,0.2)

In [ ]:
## 计算accuracy
def evaluate(W,b,lam,x_test,y_test):
    y = []
    for onehot in y_test:
        y.append(np.where(onehot > 0.5)[0][0])
    y_pre = y__dev = 1/(1+np.exp(np.dot(x_test,W)+b))
    y_ = []
    for vector in y_pre:
        y_.append(np.where(vector == max(vector))[0][0])
    loss = -np.average(y_test*np.log(y_pre)+(1-y_test)*np.log(1-y_pre))+lam/2*np.sum(W**2)
    accuracy = float(np.sum(np.array(y)==np.array(y_)))/float(len(y))
    precision = []
    recall = []
    f1 = []
    elems = list(set(y))
    for elem in elems:
        p = 0
        r = 0
        n = 0
        for i in range(len(y)):
            if (y[i] == elem):
                p += 1
            if (y_[i] == elem):
                r += 1
            if (y[i] == y_[i] and y[i] == elem):
                n +=1
        if (p != 0):
            precision.append(float(n)/float(p))
        else:
            precision.append(0)
        if (r != 0):
            recall.append(float(n)/float(r))
        else:
            recall.append(0)
    for j in range(len(precision)):
        if (precision[j]!=0 or recall[j] != 0):
            f1.append(2*precision[j]*recall[j]/(precision[j]+recall[j]))
        else:
            f1.append(0)
    return loss, accuracy,elems,precision,recall,f1

In [ ]:
## 模型评估
loss,accuracy,elems,precision,recall,f1 = evaluate(W,b,0.1,x_test,y_test)